# 3 - Faster Sentiment Analysis

In the previous notebook we managed to achieve a decent test accuracy of ~84% using all of the common techniques used for sentiment analysis. In this notebook, we'll implement a model that gets comparable results whilst training significantly faster and using around half of the parameters. More specifically, we'll be implementing the "FastText" model from the paper [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759).

## Preparing Data

One of the key concepts in the FastText paper is that they calculate the n-grams of an input sentence and append them to the end of a sentence. Here, we'll use bi-grams. Briefly, a bi-gram is a pair of words/tokens that appear consecutively within a sentence. 

For example, in the sentence "how are you ?", the bi-grams are: "how are", "are you" and "you ?".

The `generate_bigrams` function takes a sentence that has already been tokenized, calculates the bi-grams and appends them to the end of the tokenized list.

In [1]:
def generate_bigrams(x):
    n_grams = list(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

As an example:

In [2]:
generate_bigrams(['This', 'film', 'is', 'terrible'])

['This', 'film', 'is', 'terrible', 'This film', 'film is', 'is terrible']

build_dataset has `generate_bigrams` argument. 

In [3]:
import torch
from data_utils import build_dataset

train_dataset, valid_dataset = build_dataset('data/ratings_train.txt', split_ratio=0.7, generate_bigrams=True)

[Info] Get 150000 data from data/ratings_train.txt
[Info] Drop null data, now the length of this data is 149995
[Info] Tokenize...
[Info] Build vocabulary
[Info] Vocabulary size= 622857
[Info] Split 149995 data to 104996 for train data,  44999 for valid data.
[Info] Build datasets


In [4]:
assert train_dataset.vocab == valid_dataset.vocab
VOCAB = train_dataset.vocab

In [5]:
test_dataset = build_dataset('data/ratings_test.txt', predefined_vocab = VOCAB)

[Info] Get 50000 data from data/ratings_test.txt
[Info] Drop null data, now the length of this data is 49997
[Info] Tokenize...
[Info] Pre-defined vocabulary found.
[Info] Vocabulary size= 622857
[Info] Build dataset


In [6]:
print(train_dataset[0])
# print([word for word, idx in VOCAB.items() if idx in train_dataset[0][0]])

(tensor([464340,  53100, 592667, 230064, 161326, 338916, 559825]), tensor([1]))


In [7]:
torch.save((train_dataset, valid_dataset, test_dataset), 'bigram_dataset.pt')

Build the vocab and load the pre-trained word embeddings.

And create the iterators.

In [8]:
from data_utils import build_dataloader

BATCH_SIZE = 64

train_iterator = build_dataloader(train_dataset, batch_size = BATCH_SIZE)
valid_iterator = build_dataloader(valid_dataset, batch_size = BATCH_SIZE)
test_iterator = build_dataloader(test_dataset, batch_size = BATCH_SIZE)

## Build the Model

This model has far fewer parameters than the previous model as it only has 2 layers that have any parameters, the embedding layer and the linear layer. There is no RNN component in sight!

Instead, it first calculates the word embedding for each word using the `Embedding` layer (blue), then calculates the average of all of the word embeddings (pink) and feeds this through the `Linear` layer (silver), and that's it!

![](assets/sentiment8.png)

We implement the averaging with the `avg_pool2d` (average pool 2-dimensions) function. Initially, you may think using a 2-dimensional pooling seems strange, surely our sentences are 1-dimensional, not 2-dimensional? However, you can think of the word embeddings as a 2-dimensional grid, where the words are along one axis and the dimensions of the word embeddings are along the other. The image below is an example sentence after being converted into 5-dimensional word embeddings, with the words along the vertical axis and the embeddings along the horizontal axis. Each element in this [4x5] tensor is represented by a green block.

![](assets/sentiment9.png)

The `avg_pool2d` uses a filter of size `embedded.shape[1]` (i.e. the length of the sentence) by 1. This is shown in pink in the image below.

![](assets/sentiment10.png)

We calculate the average value of all elements covered by the filter, then the filter then slides to the right, calculating the average over the next column of embedding values for each word in the sentence. 

![](assets/sentiment11.png)

Each filter position gives us a single value, the average of all covered elements. After the filter has covered all embedding dimensions we get a [1x5] tensor. This tensor is then passed through the linear layer to produce our prediction.

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

As previously, we'll create an instance of our `FastText` class.

In [10]:
INPUT_DIM = len(VOCAB)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = VOCAB["<pad>"]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

Looking at the number of parameters in our model, we see we have about the same as the standard RNN from the first notebook and half the parameters of the previous model.

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 62,285,801 trainable parameters


And copy the pre-trained vectors to our embedding layer.

Not forgetting to zero the initial weights of our unknown and padding tokens.

In [12]:
UNK_IDX = VOCAB["<unk>"]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

Training the model is the exact same as last time.

We initialize our optimizer...

In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

We define the criterion and place the model and criterion on the GPU (if available)...

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

We implement the function to calculate accuracy...

In [16]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

We define a function for training our model...

**Note**: we are no longer using dropout so we do not need to use `model.train()`, but as mentioned in the 1st notebook, it is good practice to use it.

In [17]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_sentence, batch_label in iterator:
        batch_sentence = batch_sentence.to(device)
        batch_label = batch_label.to(device)
        
        optimizer.zero_grad()
        
        predictions = model(batch_sentence).squeeze(1)
        batch_label = batch_label.float()
        loss = criterion(predictions, batch_label)
        
        acc = binary_accuracy(predictions, batch_label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We define a function for testing our model...

**Note**: again, we leave `model.eval()` even though we do not use dropout.

In [18]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch_sentence, batch_label in iterator:
            batch_sentence = batch_sentence.to(device)
            batch_label = batch_label.to(device)
            
            predictions = model(batch_sentence).squeeze(1)
            batch_label = batch_label.float()
            loss = criterion(predictions, batch_label)
            
            acc = binary_accuracy(predictions, batch_label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

As before, we'll implement a useful function to tell us how long an epoch takes.

In [19]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we train our model.

In [20]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: 0.528 | Train Acc: 77.18%
	 Val. Loss: 0.405 |  Val. Acc: 83.52%
Epoch: 02 | Epoch Time: 0m 23s
	Train Loss: 0.328 | Train Acc: 87.33%
	 Val. Loss: 0.348 |  Val. Acc: 85.79%
Epoch: 03 | Epoch Time: 0m 23s
	Train Loss: 0.249 | Train Acc: 91.11%
	 Val. Loss: 0.334 |  Val. Acc: 86.46%
Epoch: 04 | Epoch Time: 0m 23s
	Train Loss: 0.193 | Train Acc: 93.45%
	 Val. Loss: 0.330 |  Val. Acc: 86.76%
Epoch: 05 | Epoch Time: 0m 23s
	Train Loss: 0.150 | Train Acc: 95.24%
	 Val. Loss: 0.337 |  Val. Acc: 86.85%


...and get the test accuracy!

The results are comparable to the results in the last notebook, but training takes considerably less time!

In [21]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.427 | Test Acc: 83.64%


## User Input

And as before, we can test on any input the user provides making sure to generate bigrams from our tokenized sentence.

In [22]:
from data_utils import tokenizer, vectorize

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok for tok in tokenizer(sentence)])
    indexed = vectorize(VOCAB, tokenized, one_sentence=True)
    tensor = torch.LongTensor(indexed).unsqueeze(0).to(device)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

An example negative review...

In [23]:
predict_sentiment(model, "이건 별로야")

0.0

An example positive review...

In [24]:
predict_sentiment(model, "이 영화는 너무 감동적이었어")

1.0

## Next Steps

In the next notebook we'll use convolutional neural networks (CNNs) to perform sentiment analysis.